In [2]:
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
import imutils 


In [3]:
img_orig = cv2.imread(r"C:\Users\Debajyoti\OneDrive\Desktop\Project 2\DocumentScanner\images\03.jpg")

In [4]:
print(img_orig.shape)

(1600, 1200, 3)


In [5]:
cv2.namedWindow('original',cv2.WINDOW_NORMAL)
cv2.imshow('original',img_orig)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
def resizer(image,width=500):
    h,w,c = image.shape
    height = int((h/w)*width)

    img = cv2.resize(image,(width,height))
    size = (width,height)
    return img,size

In [7]:
img_re,size = resizer(img_orig)
print(size)
flag = 0
cv2.imshow('Resized',img_re)
cv2.waitKey()
cv2.destroyAllWindows()
print(img_re.shape)

(500, 666)
(666, 500, 3)


In [8]:
detail = cv2.detailEnhance(img_re, sigma_s = 20, sigma_r = 0.15)#Enhance
gray = cv2.cvtColor(detail,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(5,5),0)
# edge detect
edge_image = cv2.Canny(blur,75,200)
# morphological transform
kernel = np.ones((5,5), np.uint8)
dilate = cv2.dilate(edge_image, kernel,iterations=1)
closing = cv2.morphologyEx(dilate,cv2.MORPH_CLOSE,kernel)

# find the contours
contours, hire = cv2.findContours(closing,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)

contours = sorted(contours, key = cv2.contourArea, reverse = True)
for contour in contours:
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02*peri, True)
    
    if len(approx)==4:
        four_points = np.squeeze(approx)
        break
if flag == 0:    
  cv2.drawContours(img_re,[four_points],-1,(0,0,255),3)
  flag = 1 

# find four points for original image
multiplier = img_orig.shape[1]/size[0]
four_points_orig = four_points*multiplier
four_points_orig = four_points_orig.astype(int)
cv2.imshow('resize_image',img_re)
cv2.imshow('closing',closing)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
print(four_points_orig)
print(four_points)

[[ 732  324]
 [ 199  974]
 [ 597 1296]
 [1118  640]]
[[305 135]
 [ 83 406]
 [249 540]
 [466 267]]


In [10]:
from imutils.perspective import four_point_transform


In [11]:
wrap_image = four_point_transform(img_orig,four_points_orig)


In [12]:
cv2.imshow("wrap",wrap_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
from applyBrightness import apply_brightness_contrast

magic_image = apply_brightness_contrast(wrap_image,100,100)

In [22]:
cv2.imshow("brightness imagem",magic_image)
cv2.waitKey(0)
cv2.destroyAllWindows()